In [ ]:
from pyincore import IncoreClient
from pyincore.analyses.cumulativebuildingdamage.cumulativebuildingdamage import CumulativeBuildingDamage

In [ ]:
# Connect to IN-CORE service 
client = IncoreClient()

### Cumulative Building Damage Analysis for Seaside, OR

In [ ]:
# Setting up Id's for Dataset inputs: 
# Earthquake building damage and Tsunami Building Damage
eq_bldg_dmg_id = "5c5c9686c5c0e488fcf91903"
tsunami_bldg_dmg_id = "5c5c96f6c5c0e488fcf9190f" 

In [ ]:
# Create cumulative Building Damage
cumulative_bldg_dmg = CumulativeBuildingDamage(client)

# Load input datasets
cumulative_bldg_dmg.load_remote_input_dataset("eq_bldg_dmg", eq_bldg_dmg_id)
cumulative_bldg_dmg.load_remote_input_dataset("tsunami_bldg_dmg", tsunami_bldg_dmg_id)

# Specify the result name
result_name = "cumulative_bldg_dmg_result"

# Set analysis parameters
cumulative_bldg_dmg.set_parameter("result_name", result_name)
cumulative_bldg_dmg.set_parameter("num_cpu", 4)

In [ ]:
# Run Cumulative Building Damage Analysis
cumulative_bldg_dmg.run_analysis()

In [ ]:
# Retrieve result dataset
result = cumulative_bldg_dmg.get_output_dataset("combined-result")

# Convert dataset to Pandas DataFrame
df = result.get_dataframe_from_csv()

# Display top 5 rows of output data
df.head()